# 0. Instalação de Bibliotecas

In [ ]:
import pandas as pd
import math
from numpy.random import rand
from numpy.random import seed
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import re
from functools import reduce
from sklearn.ensemble import RandomForestClassifier
from scipy.optimize import Bounds,  minimize
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import seaborn as sns
import random
import time
import os
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
import requests
#from boruta import BorutaPy
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import ExtraTreesClassifier


#1 - Criando Conjunto de Dados Simulado

## 1.1 Função geradora do conjunto de dados: A função ***gera_data_frame*** retorna um conjunto de dados sintéticos de classificação, variando os parâmetros:
- número de observações no dataframe (linhas do conjunto de dados)
- número de variáveis preditoras (colunas do conjunto de dados)
- o limite do domínio das variáveis preditoras [1;inf)
- número de periodos (safras) em que as observações foram coletadas. (Obs: default = 30, para atender TCL)
- fator de desbalanceamento da target (default = 1, retorna um desbalanceamento na casa dos 70-30, em favor da classe 0). Valor do fator varia entre (0 ; 1.1]. Quanto mais próximo ao zero, mais balanceado (50-50), quanto mais próximo ao 1.1 mais desbalanceado(99.9-0.01) em favor da classe 0.

In [ ]:
def gera_data_frame(n,p,m,q=None,d=1):
  #n = número de linhas do dataframe
  #p = número de variáveis preditoras
  #m = limite do domínio das variáveis preditoras, sendo o mínimo igual a 1
  #q = Número de safras temporais na base de dados (default é de 30 safras por TCL)
  #d = fator de desbalanceamento (default = 1, que retorna um desbalanceamento na casa dos 70-30) -> d varia entre (0 ; 1.1] (quanto mais próximo ao zero, mais balanceado, quanto mais próximo ao 1.1 mais desbalanceado)
  if q is None:
         q = n/30
  df = pd.DataFrame({'index':list(range(0,n))})
  import random
  for i in range(0,p):
    #w = random.randint(1,m)
    mu = random.randint(1,m)
    sigma = random.randint(1,m)
    w = np.random.normal(mu, sigma, n)
    df['v'+str(i)] = w
    #df['v'+str(i)] = random.choices(list(range(0, w)), k=n)
  df = df.drop(['index'],axis=1)
  vp_resposta=[]
  for z in range(0,n):
    numero_aleatorio = round(random.random())
    if (numero_aleatorio ==1) & (df.sum(axis=1)> ((df.sum(axis=1)).mean())*d)[z]:
      numero_aleatorio = 1
    else:
      numero_aleatorio = 0
    vp_resposta.append(numero_aleatorio)
  df['target'] = vp_resposta
  safras = (list(range(0,q))*int(n/q))
  safras.sort(reverse=False)
  safras = safras + [max(safras)]*(n - len(safras))
  df['safra'] = safras
  return df


### 1.1.1 Exemplo de Simulação com Dataframe de 1000 observações, 500 preditores com domínio até 200, distribuídos em 30 safras e desbalanceamento esperado na ordem de grandeza de 70-30.

In [ ]:
df = gera_data_frame(1000,500,200,30,1)
df.shape

### 1.1.2 Desbalanceamento da Target

In [ ]:
df.target.value_counts(normalize=True)

In [ ]:
df.head(5)

# 2. Desenvolvimento Método Proposto (Spearman-Markowitz-Clusterizado)

In [ ]:
entrada_CSM = time.perf_counter()

## 2.1 Criação dos Grupos de Recursos (Via Coeficiente de Correlação de Spearman)

###2.1.1 Calculando a Correlação via Spearman e Pearson das Features em relação a variável resposta

In [ ]:
#df.drop(['game_result'],axis=1).corr()
#df.corr(method= 'spearman')
target = 'target'
variaveis = df.columns.drop(['target','safra'])
correlacoes_p = []
correlacoes_s = []

for COLUNA in variaveis:
  correlacoes_p.append(np.round(df[target].corr(df[COLUNA]),2))
  correlacao, p = spearmanr(df[target], df[COLUNA])
  correlacoes_s.append(np.round(correlacao,2))
df_cor = pd.DataFrame({'Variaveis': variaveis, 'Correlacoes_S': correlacoes_s, 'Correlacoes_P': correlacoes_p})
#df_cor.sort_values(by=['Correlacoes'])
#df_cor.Correlacoes.min()
#df_cor.Correlacoes.max()
#df_cor.Correlacoes.quantile(q=0.75)
#df_cor.Correlacoes.quantile(q=0.25)
#df_cor.Correlacoes.quantile(q=0.5)


In [ ]:
df_cor.sort_values(by=['Correlacoes_S'])

In [ ]:
df_cor.sort_values(by=['Correlacoes_S'])['Correlacoes_S'].max(),df_cor.sort_values(by=['Correlacoes_S'])['Correlacoes_S'].min()

### 2.1.2 Organizando as Features em grupos com base na similaridade de Correlação de Spearman

In [ ]:
df_cor = df_cor.dropna()
df_cor['fx']= pd.qcut(df_cor['Correlacoes_P'], q=4)

lista_variaveis_fx = []
for i in list((range(0,len(pd.unique(df_cor['fx']))))):
  lista_variaveis_fx.append(df_cor[df_cor['fx']==pd.unique(df_cor['fx'])[i]].Variaveis)


tamanho_dos_grupos=[]
lista_de_grupos=[]
lista_variaveis_fx[0]
for i in list(range(0,len(lista_variaveis_fx))):
  lista_de_grupos.append('Grupo '+str(i))
  tamanho_dos_grupos.append(len(lista_variaveis_fx[i]))

divisao_grupos = pd.DataFrame({'Grupos':lista_de_grupos,'Tamanho do Grupo':tamanho_dos_grupos})
divisao_grupos


####2.1.2.3 Quantidade de Variáveis Preditoras Selecionadas:

In [ ]:
divisao_grupos['Tamanho do Grupo'].sum()

## 2.2 Ponderar a participação (importância) de cada Grupo de Recursos no treinamento do modelo (Via Teoria para Alocação de Portfólio de Markowitz)

### 2.2.1 Cálculo dos retornos (acurácia) dos grupos em cada safra

In [ ]:
y = df['target']
X_vetor = []
X_vetor_train = []
X_vetor_test = []
y_vetor_train = []
y_vetor_test = []
for i in list(range(0,len(lista_variaveis_fx))):
  X_vetor.append('X_'+str(i+1))
  X_vetor_train.append('X_train_'+str(i+1))
  X_vetor_test.append('X_test_'+str(i+1))
  y_vetor_train.append('y_train_'+str(i+1))
  y_vetor_test.append('y_test_'+str(i+1))

for i in list(range(0,len(lista_variaveis_fx))):
  X_vetor[i] = df[lista_variaveis_fx[i]]
  X_vetor_train[i], X_vetor_test[i],y_vetor_train[i], y_vetor_test[i]= train_test_split(X_vetor[i], y, test_size = 0.3)

retorno_grupo = []
for i in list(range(0,len(X_vetor))):
  df_x = X_vetor[0]
  df_x['safra']=df['safra']
  df_x['target']=df['target']
  clf = lgb.LGBMClassifier()
  retorno_safra = []
  id_safra = []
  for ANO in pd.unique(df_x['safra']):
    df_xx = df_x[df_x.safra==ANO]
    x_filtrado = df_xx.drop(['safra','target'], axis = 1)
    y_filtrado = df_xx.target
    x_filtrado_train, x_filtrado_test, y_filtrado_train, y_filtrado_test= train_test_split(x_filtrado, y_filtrado, test_size = 0.3)
    clf.fit(x_filtrado_train, y_filtrado_train)
    pred_filtrado =clf.predict(x_filtrado_test)
    retorno_safra.append(accuracy_score(pred_filtrado, y_filtrado_test))
    id_safra.append(ANO)
  df_retornos = pd.DataFrame({'Retorno Grupo '+str(i): retorno_safra, 'Safra': id_safra})
  retorno_grupo.append(df_retornos)
df_retornos = reduce(lambda  left,right: pd.merge(left,right,on=['Safra'],how='outer'), retorno_grupo)

In [ ]:
first_column = df_retornos.pop('Safra')
df_retornos.insert(0, 'Safra', first_column)
df_retornos

### 2.2.2 Construção da Fontreira eficiente (Risco vs Retorno). Em que, o risco é a variabilidade da acurácia.

In [ ]:
#Cálculo do retorno do portifolio
def get_portfolio_returns(returns, weights):
    return np.dot(returns.mean(), weights)
#Cálculo do risco do portifólio, considerando o desvio padrão
def get_portfolio_risk(returns, weights):
    covariances = returns.cov()
    var_risk = np.matmul(np.matmul((weights), covariances.values), weights)
    std_risk = np.sqrt(var_risk)
    return std_risk
def mount_portfolio(x , args):
    return get_portfolio_risk(args, x)
def compute_frontier(returns_range, returns_df):
    optimal_portfolios = []
    #restrição para que a soma dos pesos não ultrapasse o valor 1
    eq_constraint_proportions = {'type': 'eq', 'fun': lambda x: sum(x) - 1}
    #definição dos limites inferior e superior dos pesos em 0 e 1, respectivamente
    bounds = Bounds(ub=[1,1,1,1], lb=[0,0,0,0])


    #ponto de paprtida para o algoritmo de otimização, foram utilizados pesos igualmente distribuídos
    initial_guess = np.ones(len(returns_df.columns))*1/len(returns_df.columns)
    for value in returns_range:
        #restrição para garantir que o valor de retorno desejado será mantido
        eq_constraint_return = {'type':'eq', 'fun': lambda x : get_portfolio_returns(returns_df, x) - value}

        optimal_weights = minimize(mount_portfolio, x0 = initial_guess, method = 'SLSQP',
                constraints = [eq_constraint_proportions, eq_constraint_return],
                bounds=bounds,  options={'ftol':1e-9, 'disp':False},
                args = (returns_df,))
        optimal_portfolios.append(optimal_weights)

    return optimal_portfolios


In [ ]:
#Definição dos patamares de retorno máximo e mínimo
real_returns = df_retornos.drop(['Safra'],axis=1)
min_return = real_returns.mean().min()
max_return = real_returns.mean().max()
def generate_random_portfolios(num_samples, returns):
    returns_from_random_weights = []
    risks_from_random_weights = []
    weights = []
    for i in range(0, num_samples):
        random_weights = np.random.rand(len(returns.columns))
        random_weights = random_weights/np.sum(random_weights)
        weights.append(random_weights)

        random_return = get_portfolio_returns(returns, random_weights)
        random_risk = get_portfolio_risk(returns, random_weights)

        returns_from_random_weights.append(random_return)
        risks_from_random_weights.append(random_risk)
    return [risks_from_random_weights, returns_from_random_weights, weights]

In [ ]:
random_ports = generate_random_portfolios(10000, real_returns)
plt.scatter(random_ports[0], random_ports[1])
plt.ylabel('Retorno')
plt.xlabel('Risco')

### 2.2.3 Definição dos pesos de participação de cada grupo no processo de modelagem, a partir de um critério de alocação ótima (Ex: maior retorno possível no menor risco).

In [ ]:
fronteira_eficiente = pd.DataFrame({'Risco':random_ports[0],'Retorno':random_ports[1], 'Pesos':random_ports[2]})
fronteira_eficiente['Pesos'] = fronteira_eficiente.Pesos.apply(lambda x: np.round(x, 2))

#### 2.2.3.1 Binômio Risco-Retorno Ótimo (obs: Critério de variabilidade mínima, ou risco mínimo)

In [ ]:
alocacao_otima = fronteira_eficiente[fronteira_eficiente.Risco ==fronteira_eficiente.Risco.min()]
alocacao_otima[['Risco','Retorno']]

#### 2.2.3.1 Distribuição do Índice de Importância de cada Grupo (Qual a porcentagem de features contidas em cada grupo deve participar do processo de treinamento do modelo)

In [ ]:
alocacao_otima.Pesos

## 2.3 Redução aleatória dentro de cada Grupo de Features (respeitando o índice de Importância encontrado na etapa anterior).

In [ ]:
divisao_grupos['Alocacao Otima'] = alocacao_otima.Pesos.values[0]
divisao_grupos['Tamanho Final do Grupo'] = round((divisao_grupos['Tamanho do Grupo']*divisao_grupos['Alocacao Otima'])/divisao_grupos['Alocacao Otima'].sum(),0)
divisao_grupos

In [ ]:
print('Número final de váriaveis: ',divisao_grupos['Tamanho Final do Grupo'].sum()),
print('Isso equivale à ',100*(round(1 -(divisao_grupos['Tamanho Final do Grupo'].sum()/divisao_grupos['Tamanho do Grupo'].sum()),2)),'% de redução no volume das variáveis inicias')

In [ ]:
variaveis_filtradas_csm = []
for i in list(range(0,len(X_vetor))):
  df_x = X_vetor[i]
  df_x = df_x.drop(['safra','target'],axis=1,errors='ignore')
  df_x = df_x.sample(n=int(divisao_grupos['Tamanho Final do Grupo'][i]),axis='columns')
  #df_x = df_x.sample(n=math.ceil(divisao_grupos['Alocacao Otima'][i]*df_x.shape[1]),axis='columns')
  variaveis_filtradas_csm.extend(df_x.columns)

In [ ]:
saida_CSM = time.perf_counter()
tempo_CSM = saida_CSM - entrada_CSM
'O Projeto levou '+str(round(tempo_CSM,0))+' segundos para rodar até essa fase.'

__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

# 3. Implantação de Métodos Desafiantes:
- Método de Filtragem por Informação Mútua
- Método *Embedded* por Algorítmo ExtraTrees
- Método Híbrido combinando Informação Mútua e Extratrees

## 3.1 Método de Seleção Embedded via Algoritmo de Árvores Extremamente Aleatórias (ExtraTrees)

In [ ]:
entrada_ExtraTrees = time.perf_counter()
def funcao_variaveis_importantes(DADOS,varesp = 'y'):
        DADOS = DADOS.drop(['safra'], axis=1)
        X = DADOS.loc [:, DADOS.columns != varesp]
        Y = DADOS.loc [:, DADOS.columns == varesp]
        warnings.filterwarnings("ignore")
        model = ExtraTreesClassifier()
        model.fit(X,Y)
        ind = X.shape[1]
        variaveis = X.columns[0:ind]
        importancia = pd.DataFrame(variaveis)
        features = pd.DataFrame(model.feature_importances_)
        importancia['Importancia']= features
        importancia = importancia.rename(columns = {0:"Variáveis","Importancia":"Importancia"})
        return importancia.sort_values(['Importancia'], ascending=False)

lista_vp_ExtraTrees = funcao_variaveis_importantes(df,varesp = 'target')
X_filtrado_ExtraTrees = df[lista_vp_ExtraTrees[lista_vp_ExtraTrees.Importancia>0]['Variáveis'].values] #Importância maior que zero
lista_vp_filtrada_ExtraTrees = X_filtrado_ExtraTrees.columns
saida_ExtraTrees = time.perf_counter()
tempo_ExtraTrees = saida_ExtraTrees - entrada_ExtraTrees
'O algoritmo levou '+str(round(tempo_ExtraTrees,0))+' segundos para rodar e deixou '+str(len(lista_vp_filtrada_ExtraTrees))+' variáveis preditoras.'

## 3.2 Método de Seleção por Filtro via  Informação Mútua

In [ ]:
entrada_MI = time.perf_counter()
variaveis = df.columns.drop(['target','safra'])
X_train_mutual_info, X_test_mutual_info,y_train_mutual_info, y__test_mutual_info= train_test_split(df[variaveis], df[target].to_numpy(), test_size = 0.3)
mutual_info = mutual_info_classif(X_train_mutual_info, y_train_mutual_info)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train_mutual_info.columns
lista_vp_filtrada_MI = mutual_info[mutual_info.sort_values(ascending=False)>0] #Ganho de Informação maior que zero
saida_MI = time.perf_counter()
tempo_MI = saida_MI - entrada_MI
'O algoritmo levou '+str(round(tempo_MI,0))+' segundos para rodar e deixou '+str(len(lista_vp_filtrada_MI))+' variáveis preditoras.'

## 3.3 Método Combinado de Seleção (Informação Mutua e ExtraTrees)

In [ ]:
entrada_COMBINADO = time.perf_counter()
df_combinado = df[lista_vp_filtrada_MI.index]
df_combinado['target'] = df.target
def funcao_variaveis_importantes(DADOS,varesp = 'y'):
        X = DADOS.loc [:, DADOS.columns != varesp]
        Y = DADOS.loc [:, DADOS.columns == varesp]
        warnings.filterwarnings("ignore")
        model = ExtraTreesClassifier()
        model.fit(X,Y)
        ind = X.shape[1]
        variaveis = X.columns[0:ind]
        importancia = pd.DataFrame(variaveis)
        features = pd.DataFrame(model.feature_importances_)
        importancia['Importancia']= features
        importancia = importancia.rename(columns = {0:"Variáveis","Importancia":"Importancia"})
        return importancia.sort_values(['Importancia'], ascending=False)

lista_vp_COMBINADO = funcao_variaveis_importantes(df_combinado,varesp = 'target')
X_filtrado_COMBINADO = df_combinado[lista_vp_COMBINADO[lista_vp_COMBINADO.Importancia>0]['Variáveis'].values] #Importância maior que zero
lista_vp_filtrada_COMBINADO = X_filtrado_COMBINADO.columns
saida_COMBINADO = time.perf_counter()
tempo_COMBINADO = saida_COMBINADO - entrada_COMBINADO
'O algoritmo levou '+str(round(tempo_COMBINADO,0))+' segundos para rodar e deixou '+str(len(lista_vp_filtrada_COMBINADO))+' variáveis preditoras.'

# 4. Comparação do Desempenho dos Algoritmos: Análise se a exclusão de features resultou em perda significativa de performance.
- Modelo construído com a base completa (Sem processo de seleção de feature)
- Modelo construído com a base selecionada a partir do Método de Informação Mútua
- Modelo construído com a base selecionada a partir do Método de Seleção ExtraTrees
- Modelo construído com base selecionada a partir da combinação entre Informação Mutua e ExtraTrees
- Modelo construído com base selecionada a partir do Método Proposto (Spearman-Markowitz Clusterizado)

Obs: Para evitar viés do algoritmo, todos os modelos foram criados a partir do algoritmo LightGBM

## 4.1 Acurácia estimada a partir de Modelo construído com a Base Completa

In [ ]:
  entrada_FULL = time.perf_counter()
  X_train, X_test,y_train, y__test= train_test_split(df[variaveis], df[target], test_size = 0.3)
  clf = lgb.LGBMClassifier()
  clf.fit(X_train, y_train)
  predicao = clf.predict(X_test)
  acuracia_total_FULL = accuracy_score(predicao, y__test)
  saida_FULL = time.perf_counter()
  tempo_FULL = saida_FULL - entrada_FULL
  'O algoritmo levou '+str(round(tempo_FULL,0))+' segundos para rodar um modelo com todas as variáveis preditoras, obtedo uma acurácia de '+ str(round(acuracia_total_FULL,3))+'.'

##4.2 Acurácia estimada a partir de Modelo construído com a base selecionada a partir do Método de Informação Mútua

In [ ]:
entrada_MI_model= time.perf_counter()
sel_cols = SelectKBest(mutual_info_classif, k=len(lista_vp_filtrada_MI))
sel_cols.fit(X_train_mutual_info, y_train_mutual_info)
X_filtrado = df[X_train_mutual_info.columns[sel_cols.get_support()]]
X_train, X_test,y_train, y_test= train_test_split(X_filtrado, df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
predicao = clf.predict(X_test)
acuracia_total_MI_model = accuracy_score(predicao, y_test)
saida_MI_model = time.perf_counter()
tempo_MI_model = saida_MI_model - entrada_MI_model
'O processo de Modelagem levou '+str(round(tempo_MI_model,0))+' segundos para rodar com '+str(len(lista_vp_filtrada_MI))+' variáveis preditoras, resultando em uma acurácia de '+ str(round(acuracia_total_MI_model,3))+'.'


## 4.3 Acurácia estimada a partir de Modelo construído com a base selecionada a partir do Método *Embedeed* via Algoritmo de Árvores Extremamente Aleatórias.

In [ ]:
entrada_ExtraTree_Model = time.perf_counter()

X_train, X_test,y_train, y_test= train_test_split(X_filtrado_ExtraTrees, df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
predicao = clf.predict(X_test)
acuracia_ExtraTree_Model = accuracy_score(predicao, y_test)
saida_ExtraTree_Model = time.perf_counter()
tempo_ExtraTree_Model = saida_ExtraTree_Model - entrada_ExtraTree_Model
'O processo de Modelagem levou '+str(round(tempo_ExtraTree_Model,0))+' segundos para rodar com '+str(len(lista_vp_filtrada_ExtraTrees))+' variáveis preditoras, resultando em uma acurácia de '+ str(round(acuracia_ExtraTree_Model,3))+'.'


## 4.4 Acurácia estimada a partir de Modelo construído com a base selecionada a partir do Método combinado entre a Informação Mútua e o Algoritmo de Árvores Extremamente Aleatórias.

In [ ]:
entrada_combinado_Model = time.perf_counter()

X_train, X_test,y_train, y_test= train_test_split(X_filtrado_COMBINADO, df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
predicao = clf.predict(X_test)
acuracia_combinado_Model = accuracy_score(predicao, y_test)
saida_combinado_Model = time.perf_counter()
tempo_combinado_Model = saida_combinado_Model - entrada_combinado_Model
'O processo de Modelagem levou '+str(round(tempo_combinado_Model,0))+' segundos para rodar com '+str(len(lista_vp_filtrada_ExtraTrees))+' variáveis preditoras, resultando em uma acurácia de '+ str(round(acuracia_combinado_Model,3))+'.'


## 4.5 Acurácia estimada a partir de Modelo construído com a base selecionada a partir do Método Spearman-Markowitz Clusterizado.

In [ ]:
entrada_CSM_model = time.perf_counter()
X_train_final, X_test_final,y_train_final, y_test_final= train_test_split(df[variaveis_filtradas_csm], df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train_final, y_train_final)
predicao = clf.predict(X_test_final)
acuracia_CSM_model = accuracy_score(predicao, y_test_final)
saida_CSM_model = time.perf_counter()
tempo_CSM_model = saida_CSM_model - entrada_CSM_model
'A proposta levou '+str(round(tempo_CSM_model,0))+' segundos para rodar com '+str(len(variaveis_filtradas_csm))+' variáveis preditoras, resultando em uma acurácia de '+ str(round(acuracia_CSM_model,3))+'.'

# 5. Tabela Resumo de desempenhos:


In [ ]:
Metodo_de_Seleção = ['Sem Seleção','Informação Mutua', 'ExtraTrees','Combinado', 'CSM']
N_features = [df.shape[1]-2,len(lista_vp_filtrada_MI),len(lista_vp_filtrada_ExtraTrees),len(lista_vp_filtrada_COMBINADO),len(variaveis_filtradas_csm)]
Performance = [round(acuracia_total_FULL,3),round(acuracia_total_MI_model,3),round(acuracia_ExtraTree_Model,3),round(acuracia_combinado_Model,3),round(acuracia_CSM_model,3)]
Tempo_modelagem = [round(tempo_FULL,3),round(tempo_MI_model,3),round(tempo_ExtraTree_Model,3),round(tempo_combinado_Model,3),round(tempo_CSM_model,3)]
Tempo_selecao = [0,round(tempo_MI,3),round(tempo_ExtraTrees,3),round(tempo_COMBINADO,3),round(tempo_CSM,3)]

resultados = pd.DataFrame({'Métodos de Seleção':Metodo_de_Seleção,'N° de Features':N_features,'Acurácia do Modelo Final':Performance,
              'Tempo de Treinamento do Modelo':Tempo_modelagem,'Tempo do Processamento da Seleção de Feat.':Tempo_selecao})

resultados.sort_values(by=['Acurácia do Modelo Final'], ascending=False)

,Métodos de Seleção,N° de Features,Acurácia do Modelo Final,Tempo de Treinamento do Modelo,Tempo do Processamento da Seleção de Feat.
2,ExtraTrees,500,0.777,4.460,0.523
3,Combinado,235,0.777,1.974,0.363
4,CSM,120,0.770,0.951,20.075
0,Sem Seleção,500,0.763,4.085,0.000
1,Informação Mutua,235,0.750,4.562,2.181
